In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.datasets import load_files
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SimpleRNN
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.engine import Input
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import KeyedVectors
import matplotlib as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Training data**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/malayalam_news_train-1.csv')

In [ ]:
#Converting the dataframe into list
headings = data['headings'].tolist()
genere = data['label'].tolist()

corpus = []
for i in range(len(headings)):
    corpus.append(word_tokenize(headings[i]))

**Feature extraction**

In [ ]:
model_w2v = Word2Vec(corpus, size=100, window=5, min_count=1)

In [ ]:
model_w2v.wv.vocab

{'കൊച്ചി': <gensim.models.keyedvectors.Vocab at 0x7f0676beb470>,
 'ചെരുപ്പ്': <gensim.models.keyedvectors.Vocab at 0x7f0676beb4a8>,
 'വിതരണ': <gensim.models.keyedvectors.Vocab at 0x7f0676beb278>,
 'കേന്ദ്രത്തിലെ': <gensim.models.keyedvectors.Vocab at 0x7f0676beb2b0>,
 'തീപിടിത്തം': <gensim.models.keyedvectors.Vocab at 0x7f0676beb2e8>,
 ';': <gensim.models.keyedvectors.Vocab at 0x7f0676beb320>,
 'അന്വേഷണം': <gensim.models.keyedvectors.Vocab at 0x7f0676beb358>,
 'തുടങ്ങി': <gensim.models.keyedvectors.Vocab at 0x7f0676beb390>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7f0676beb3c8>,
 'അട്ടിമറി': <gensim.models.keyedvectors.Vocab at 0x7f0676beb400>,
 'സാധ്യതയും': <gensim.models.keyedvectors.Vocab at 0x7f0676beb438>,
 'പരിശോധിക്കുന്നു': <gensim.models.keyedvectors.Vocab at 0x7f0676beb518>,
 'ഈ\u200b\u200b\u200bസ്റ്റേ\u200b\u200b\u200bണ്\u200d': <gensim.models.keyedvectors.Vocab at 0x7f0676beb550>,
 'ക\u200b\u200b\u200bറി': <gensim.models.keyedvectors.Vocab at 0x7f0676beb588>,
 'പൗ\u200b\

In [ ]:
#Tokenization
max_features = 2000 #number of words to keep. 
#2000 is the number of unique words in the corpus.
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(data['headings'].values)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [ ]:
#Unique words and and their count
tokenizer.word_counts

OrderedDict([('കൊച്ചി', 6),
             ('ചെരുപ്പ്', 1),
             ('വിതരണ', 5),
             ('കേന്ദ്രത്തിലെ', 1),
             ('തീപിടിത്തം', 1),
             ('അന്വേഷണം', 6),
             ('തുടങ്ങി', 41),
             ('അട്ടിമറി', 3),
             ('സാധ്യതയും', 1),
             ('പരിശോധിക്കുന്നു', 1),
             ('ഈ\u200b\u200b\u200bസ്റ്റേ\u200b\u200b\u200bണ്\u200d', 1),
             ('ക\u200b\u200b\u200bറി', 1),
             ('പൗ\u200b\u200b\u200bഡ\u200b\u200b\u200bറി\u200b\u200b\u200bന്',
              1),
             ('പു\u200b\u200b\u200bര\u200b\u200b\u200bസ്കാ\u200b\u200b\u200bരം',
              1),
             ('സം\u200b\u200b\u200bസ്ഥാ\u200b\u200b\u200bന', 1),
             ('മ\u200b\u200b\u200bലി\u200b\u200b\u200bനീ\u200b\u200b\u200bക\u200b\u200b\u200bര\u200b\u200b\u200bണ',
              1),
             ('നി\u200b\u200b\u200bയ\u200b\u200b\u200bന്ത്ര\u200b\u200b\u200bണ',
              1),
             ('ബോ\u200b\u200b\u200bര്\u200d\u200b\u200b\u200bഡി\u200b\u200b\u200

In [ ]:
X = tokenizer.texts_to_sequences(data['headings'].values)
X = pad_sequences(X, padding = 'post') #Zero padding at the end of the sequence

In [ ]:
labels = data['label'].tolist()
le = preprocessing.LabelEncoder()
Y = le.fit_transform(labels)

In [ ]:
Y = to_categorical(Y)
print(Y)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
#Splitting the data into train data and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
print("Shape train data = ",np.shape(X_train))
print("Shape of train label = ",np.shape(Y_train))
print("Shape of test data = ",np.shape(X_test))
print("Shape of test label = ",np.shape(Y_test))

Shape train data =  (4028, 25)
Shape of train label =  (4028, 3)
Shape of test data =  (1008, 25)
Shape of test label =  (1008, 3)


**ARCHITECTURE**

In [ ]:
embed_dim=500
model=Sequential()
model.add(Embedding(max_features,embed_dim,input_length=X.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 25, 500)           900000    
_________________________________________________________________
dropout_7 (Dropout)          (None, 25, 500)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               240400    
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 303       
Total params: 1,140,703
Trainable params: 1,140,703
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,Y_train,epochs = 20, batch_size = 32)

Epoch 1/20
126/126 [==============================] - 15s 116ms/step - loss: 0.9280 - accuracy: 0.5032
Epoch 2/20
126/126 [==============================] - 16s 124ms/step - loss: 0.3418 - accuracy: 0.8784
Epoch 3/20
126/126 [==============================] - 15s 119ms/step - loss: 0.1875 - accuracy: 0.9295
Epoch 4/20
126/126 [==============================] - 15s 116ms/step - loss: 0.1625 - accuracy: 0.9402
Epoch 5/20
126/126 [==============================] - 15s 116ms/step - loss: 0.1408 - accuracy: 0.9451
Epoch 6/20
126/126 [==============================] - 15s 116ms/step - loss: 0.1368 - accuracy: 0.9469
Epoch 7/20
126/126 [==============================] - 15s 117ms/step - loss: 0.1303 - accuracy: 0.9469
Epoch 8/20
126/126 [==============================] - 16s 126ms/step - loss: 0.1032 - accuracy: 0.9551
Epoch 9/20
126/126 [==============================] - 15s 118ms/step - loss: 0.1235 - accuracy: 0.9503
Epoch 10/20
126/126 [==============================] - 15s 117ms/step - l

**Validation Accuracy**

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

32/32 [==============================] - 1s 18ms/step - loss: 0.4238 - accuracy: 0.8929
Test set
  Loss: 0.424
  Accuracy: 0.893


In [ ]:
import matplotlib as plt

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='Validation')
plt.legend()
plt.show();

AttributeError: ignored

**Loading Test Data**

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/S3/NLP/CONTEST/malayalam_news_test.csv')
test.head()

,ID,headings
0,MAL_0001,അന്താരാഷ്ട്ര ഇന്റര്‍നെറ്റ് സേവനം താല്‍ക്കാലികമ...
1,MAL_0002,"മഞ്ഞ് ചതിച്ചാശാനേ!!!!, ഹാന്നോവറിന്റെ പ്രതീക്ഷക..."
2,MAL_0003,"12 സീസണ്‍ എട്ടു ചാമ്ബ്യന്മാര്‍, പ്രവചിക്കാന്‍ ..."
3,MAL_0004,\nമുംബൈ സിറ്റിയോടു തോറ്റിട്ടും യോഗ്യത നേടി ഗോവ...
4,MAL_0005,മദ്യപിച്ചെത്തിയ നടന്മാര്‍ തമ്മില്‍ കയ്യാങ്കളി;...


In [ ]:
test = tokenizer.texts_to_sequences(test['headings'].values)
test = pad_sequences(test, padding = 'post')

In [ ]:
class_label = model.predict_classes(test)
print(le.inverse_transform(class_label))

['business' 'business' 'sports' ... 'business' 'business' 'business']


In [ ]:
pred_labels=le.inverse_transform(class_label)
print(pred_labels)

['business' 'business' 'sports' ... 'business' 'business' 'business']


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/S3/NLP/CONTEST/ROLLNO_Task_1_submission.csv')
df.head()

,ID,Predicted labels
0,MAL_0001,NaN
1,MAL_0002,NaN
2,MAL_0003,NaN
3,MAL_0004,NaN
4,MAL_0005,NaN


In [ ]:
df['Predicted labels']=pred_labels

In [ ]:
pred_labels

array(['business', 'business', 'sports', ..., 'business', 'business',
       'business'], dtype='<U13')

In [ ]:
df.to_csv(r'/content/drive/MyDrive/S3/NLP/CONTEST/pred_2.csv', index=False)

In [ ]:
df.head()

,ID,Predicted labels
0,MAL_0001,business
1,MAL_0002,business
2,MAL_0003,sports
3,MAL_0004,sports
4,MAL_0005,entertainment


In [ ]:
len(df)

1260

** Both RNN nad LSTM , the validation accuracy obtained in case of LSTM is 89.3 and in RNN is 85.4**  
**LSTM provived better text classification in this case**